In [19]:
!git clone https://github.com/yk03studios-wq/AnimeGANv3-fixed.git
%cd AnimeGANv3-fixed

In [20]:
!pip install -q onnxruntime-gpu tqdm Pillow
!pip install -q -r requirements.txt

In [21]:
!sed -i '/onnxruntime==/d' requirements.txt
!sed -i '/onnxruntime/d' requirements.txt
!sed -i '/tensorflow/d' requirements.txt

In [22]:
!pip install -q onnxruntime==1.22.1

In [23]:
!pip install -q -r requirements.txt

In [24]:
import onnxruntime as ort
print("ONNX Runtime:", ort.__version__)
print("Providers:", ort.get_available_providers())

In [ ]:
!python tools/video2anime.py \
-i inputs/vid/1.mp4 \
-o output/results \
-m deploy/AnimeGANv3_Hayao_36.onnx

In [26]:
!ls -la output/results/

In [31]:
import os
import json
import subprocess
from pathlib import Path

OUTPUT_DIR = Path("/kaggle/working/output/results")
DATASET_DIR = Path("/kaggle/working/dataset_export")

DATASET_SLUG = "animegan-outputs-latest"
DATASET_OWNER = "yashkawade03"
DATASET_ID = f"{DATASET_OWNER}/{DATASET_SLUG}"

# Safety check
if not OUTPUT_DIR.exists():
    raise RuntimeError(f"Output directory not found: {OUTPUT_DIR}")

# Prepare export directory
DATASET_DIR.mkdir(parents=True, exist_ok=True)

# Copy outputs
for file in OUTPUT_DIR.iterdir():
    if file.is_file():
        target = DATASET_DIR / file.name
        if target.exists():
            target.unlink()
        target.write_bytes(file.read_bytes())

print(f"Copied outputs to {DATASET_DIR}")

# Create dataset metadata
dataset_metadata = {
    "title": "AnimeGAN Outputs (Latest)",
    "id": DATASET_ID,
    "licenses": [
        {
            "name": "CC0-1.0"
        }
    ]
}

metadata_path = DATASET_DIR / "dataset-metadata.json"
with open(metadata_path, "w") as f:
    json.dump(dataset_metadata, f, indent=2)

print("dataset-metadata.json written")

# Push dataset (create or update)
cmd = [
    "kaggle", "datasets", "create",
    "-p", str(DATASET_DIR),
    "--dir-mode", "zip",
    "--force"
]

print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

print(f"✅ Dataset updated: https://www.kaggle.com/datasets/{DATASET_ID}")